In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from keras.callbacks import CSVLogger
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime

!rm -rf ./logs/GRU

tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [3]:
#retrieve and format data - into labels from the dataset
def labels(filename):
    data = np.matrix(pd.read_csv(filename).values)
    data_y = data[:, 0]
    data_y-=1 
    return data_y

In [4]:
with open("../archive/ProcessData.npy", 'rb') as f:
        training_data_x = np.load(f)
        test_x = np.load(f)
training_data_x = convert_3D(training_data_x) 
test_x = convert_3D(test_x)

training_data_y = labels("../archive/exoTrain.csv")
test_y = labels("../archive/exoTest.csv")

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

In [5]:
GRUmodel = Sequential([
    GRU(64, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    #Dropout(0.3),
    GRU(64),
    #Dropout(0.3),
    #GRU(64),
    Dense(1, activation='sigmoid')
])

GRUmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy"]
)

log_dir = "logs/GRU/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

csv_logger = CSVLogger('keras-models/KerasGRU/training.csv',separator=",", append=False)

GRUmodel.fit(training_data_x, training_data_y, epochs=10, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = GRUmodel
model.save('keras-models/KerasGRU')

Epoch 1/10
112/112 [==============================] - 28s 207ms/step - loss: 0.6641 - accuracy: 0.6795 - val_loss: 0.6279 - val_accuracy: 0.9790
Epoch 2/10
112/112 [==============================] - 20s 182ms/step - loss: 0.5979 - accuracy: 0.9685 - val_loss: 0.5649 - val_accuracy: 0.9574
Epoch 3/10
112/112 [==============================] - 20s 182ms/step - loss: 0.5360 - accuracy: 0.9573 - val_loss: 0.5044 - val_accuracy: 0.9509
Epoch 4/10
112/112 [==============================] - 20s 183ms/step - loss: 0.4753 - accuracy: 0.9534 - val_loss: 0.4444 - val_accuracy: 0.9509
Epoch 5/10
112/112 [==============================] - 20s 181ms/step - loss: 0.4147 - accuracy: 0.9539 - val_loss: 0.3854 - val_accuracy: 0.9528
Epoch 6/10
112/112 [==============================] - 20s 183ms/step - loss: 0.3548 - accuracy: 0.9565 - val_loss: 0.3275 - val_accuracy: 0.9561
Epoch 7/10
112/112 [==============================] - 20s 182ms/step - loss: 0.2971 - accuracy: 0.9587 - val_loss: 0.2733 - val_ac

INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


INFO:tensorflow:Assets written to: keras-models/KerasGRU/assets


In [6]:
%tensorboard --logdir logs/GRU/fit